In [6]:
%matplotlib qt
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as mpl
from pathlib import Path
import pandas as pd
import pynwb as NWB

fn = Path('VCN_c06/Simulations/IV/VCN_c06_pulse_XM13_nacncoop_II_monitor.p')
fn = Path('VCN_c06/Simulations/IV/VCN_c06_pulse_XM13_nacncoop_II_monitor_noDendNa_Full.p')
fn = Path('VCN_c11/Simulations/AN/AN_Result_VCN_c11_inp=self_XM13_nacncoop_II_soma=1.203_dend=1.510_ASA=1.203_delays_multisite_001_tonepip_050dB_4000.0_HS.p.dat')
basepath = Path('/Users/pbmanis/Desktop/Python/VCN-SBEM-Data/VCN_Cells')
fnb = Path(basepath, fn)
print(fnb.is_file())

d = pd.read_pickle(fnb)


True


In [17]:
print(d['trials'][0].keys())

dict_keys(['stimInfo', 'spikeTimes', 'inputSpikeTimes', 'time', 'somaVoltage', 'dendriteVoltage', 'allDendriteVoltages', 'stimWaveform', 'stimTimebase'])


In [7]:
"""
Plot all locations for one stimulus (IV data)
"""
f, ax = mpl.subplots(1,1)
spines_to_remove = ['top', 'right']
ax = [ax]
for a in ax:
    for spine in spines_to_remove:
        a.spines[spine].set_visible(False)
for i, R in enumerate(d['Results']):
    if i < len(d['Results'])-1:
        continue
#     print(R)
    inj = list(R.keys())[0]
    r = R[inj]
    for v in r['vec']:
        ax[0].plot(r['monitor']['time'], r['vec'][v])
mpl.show()

KeyError: 'Results'

In [4]:
"""
Save IV data to csv file with time on left column and sections ordered to the right
we do this by building the csv as a pandas data frame and filling it
"""
for i, R in enumerate(d['Results']):
    if i < len(d['Results'])-1:
        continue
#     print(R)
    inj = list(R.keys())[0]
    r = R[inj]
    secnos = len(r['vec'])
    secs = ['time']
    for s in range(secnos):
        secs.append(f'sections[{s:d}]')
    df = pd.DataFrame(columns=secs)
    print(df.columns)
    df['time'] = r['monitor']['time']
    for v in secs[1:]:
        df[v] = r['vec'][v]

df.to_csv(Path('..', fn.name).with_suffix('.csv'))
df.to_json(Path('..', fn.name).with_suffix('.json'))
df.to_hdf(Path('..', fn.name).with_suffix('.hdf'), key='Voltage')
df.to_parquet(Path('..', fn.name).with_suffix('.parquet'))



Index(['time', 'sections[0]', 'sections[1]', 'sections[2]', 'sections[3]',
       'sections[4]', 'sections[5]', 'sections[6]', 'sections[7]',
       'sections[8]',
       ...
       'sections[265]', 'sections[266]', 'sections[267]', 'sections[268]',
       'sections[269]', 'sections[270]', 'sections[271]', 'sections[272]',
       'sections[273]', 'sections[274]'],
      dtype='object', length=276)


/Users/pbmanis/anaconda/envs/py3mpl3/lib/python3.6/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


In [35]:
"""
Plot all locations for one stimulus (AN data)
Each trial contains a dict:
dict_keys(['stimInfo', 'spikeTimes', 'inputSpikeTimes', 'time',
'somaVoltage', 'dendriteVoltage', 'allDendriteVoltages', 
'stimWaveform', 'stimTimebase'])

"""
f, ax = mpl.subplots(1,1)
spines_to_remove = ['top', 'right']
ax = [ax]
for a in ax:
    for spine in spines_to_remove:
        a.spines[spine].set_visible(False)
for trial in d['trials']:
    dx = d['trials'][trial]
    print('trial: ', trial)
    time = dx['time']
    for tx in dx['allDendriteVoltages']:
        for v in dx['allDendriteVoltages'][tx]:
            ax[0].plot(time, dx['allDendriteVoltages'][tx][v], linewidth=0.25)
mpl.show()

trial:  0


In [ ]:
"""
Save AN data to NWB formatted file
"""

        subject = NWB.file.Subject(age=0, 
                description='vcnmodel', genotype='None', sex='None', species='Computer', 
                subject_id=None, weight='None')

    #     print(info)
        nwbfile = NWB.NWBFile('LDA Data set', str(datafilename), datetime.datetime.now(tzlocal()),
                      experimenter='Manis, Paul',
                      lab='Manis Lab',
                      institution='UNC Chapel Hill',
                      experiment_description='Model Output',
                      session_id=f"{i:d}",
                      notes=f"Cell Type: Bushy XM13_nacncoop AN",
                      subject=subject)
        device = NWB.device.Device('MC700B', parent=None)
        nwbfile.add_device(device)
        elec = NWB.icephys.IntracellularElectrode(name="elec0",
                                       description='KG33 patch as intracellular electrode',
                                       device=device)
        nwbfile.add_ic_electrode(elec)  # not well documented!

        istim = NWB.icephys.CurrentClampStimulusSeries( name="Ics", data=np.array(cmd_wave), unit='A',
                starting_time=info['__timestamp__'], rate=sample_rate[0],
                electrode=elec, gain=1.0, sweep_number=1)
        vdata = NWB.icephys.CurrentClampSeries('Vcs', data=data_array, 
            unit='V', electrode=elec, gain=1.0, bias_current=0.0, bridge_balance=0., 
            capacitance_compensation=0., stimulus_description='NA', 
            resolution=0.0, conversion=1.0, timestamps=None, 
            starting_time=info['__timestamp__'], rate=sample_rate[0], 
            comments='no comments', description='no description', 
            control=None, control_description=None, sweep_number=1, parent=None)
        nwbfile.add_acquisition(istim)
        nwbfile.add_acquisition(vdata)

        with NWB.NWBHDF5IO(str(Path(opath, outfilename))+'.nwb', 'w') as io:
            io.write(nwbfile)
        nwrite += 1
print('ok')